In [5]:
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias

df_train = pd.read_csv('./data/train.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features_de_csvs(df_train, df_eval)

# Randoms solo para asegurarse que los features esten bien hechos
# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_train = utils.dolarizar_df(df_train)
# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_test = utils.pesificar_df(df_test)
# df_train = df_train.sample(frac=1).reset_index(drop=True)


In [6]:
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train = features.llenar_nulls(df_train, hgb_mean=True)

In [7]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_f = df_train_f.sample(frac=1).reset_index(drop=True)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)



In [ ]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, AlphaDropout

def normalizar_df(df, features):
    min_max = StandardScaler()
    df[features] = pd.DataFrame(min_max.fit_transform(df[features]), columns=features)
    return df
    


features = ['habitaciones', 'garages','banos','antiguedad', 'metroscubiertos',  'metrostotales','lat_norm', 
           'lng_norm', 'gimnasio', 'usosmultiples', 'piscina']

features_test = ['top_provincia', 'promedio_precio_ciudad', 'anio', 'promedio_id_zona', 
                 'promedio_precio_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
               'count_tipo_propiedad_ciudad', 'promedio_precio_tipo_propiedad_ciudad_gen','count_id_zona',
           'dias_desde_datos','meses_desde_datos','porcentaje_metros','distancia_ciudad_centrica', 'puntaje', 'distancia_centro_mexico']


features_cat = ['provincia', 'tipodepropiedad', 'intervalo_metros_totales', 'intervalo_metros_cubiertos',
               'zona']

features += features_test + features_cat

df_train_g = utils.filtrar_features(df_train_f, features, 'precio')

features_a_normalizar = [f for f in features if f not in features_cat + ['precio']]

df_train_n = pd.get_dummies(df_train_g, columns=features_cat)
df_train_n = normalizar_df(df_train_n, features_a_normalizar)

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_n, 'precio', test_size=0.001)

model = Sequential()
model.add(Dense(units=200, activation='selu'))
model.add(Dropout(0.1))
model.add(Dense(units=200, activation='selu'))
model.add(Dropout(0.1))
model.add(Dense(units=200, activation='selu'))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])

model.fit(x_train.values, y_train.values, epochs=10)

y_pred_train = model.predict(x_train.values)
y_pred_test = model.predict(x_test)

keras_mae_train = utils.MAE(y_train.values, y_pred_train)
keras_mae_test = utils.MAE(y_test, y_pred_test)
print(f"MAE Keras (train): {keras_mae_train:.5f}")
print(f"MAE Keras (test): {keras_mae_test:.5f}")


Epoch 1/10
203796/203796 [==============================] - 23s 114us/step - loss: 754908.8148 - mean_squared_error: 1772677234688.0000
Epoch 2/10
203796/203796 [==============================] - 24s 117us/step - loss: 617401.9226 - mean_squared_error: 1137125359616.0000
Epoch 3/10
203796/203796 [==============================] - 25s 125us/step - loss: 602291.1683 - mean_squared_error: 1093779259392.0000
Epoch 4/10
203796/203796 [==============================] - 25s 123us/step - loss: 593689.3709 - mean_squared_error: 1068598296576.0000
Epoch 5/10
203796/203796 [==============================] - 25s 122us/step - loss: 586479.0589 - mean_squared_error: 1047404085248.0000
Epoch 6/10
203796/203796 [==============================] - 26s 127us/step - loss: 582041.9311 - mean_squared_error: 1033088860160.0000
Epoch 7/10
203796/203796 [==============================] - 24s 117us/step - loss: 578811.3998 - mean_squared_error: 1021025583104.0000
Epoch 8/10
203796/203796 [======================

In [27]:
df_train_n.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'gimnasio', 'usosmultiples', 'piscina', 'precio',
       'porcentaje_metros', 'top_provincia', 'anio', 'dias_desde_datos',
       'meses_desde_datos', 'promedio_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad_gen',
       'count_tipo_propiedad_ciudad', 'lat_norm', 'lng_norm', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'provincia_Aguascalientes', 'provincia_Baja California Norte',
       'provincia_Baja California Sur', 'provincia_Campeche',
       'provincia_Chiapas', 'provincia_Chihuahua', 'provincia_Coahuila',
       'provincia_Colima', 'provincia_Distrito Federal', 'provincia_Durango',
       'provincia_Edo. de México', 'provincia_Guanajuato',
       'provincia_Guerrero', 'provincia_Hidalgo', 'provincia_Jalisco',
       'provincia_Michoacán'

In [ ]:

import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio, features_de_csvs

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
    
def normalizar_df(df, features):
    min_max = StandardScaler()
    df[features] = pd.DataFrame(min_max.fit_transform(df[features]), columns=features)
    return df

def llenar_nulls(df):
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metrostotales'].fillna(df['metrostotales'])
    
    df['habitaciones'] = df['habitaciones'].fillna(df['habitaciones'].mean())
    df['garages'] = df['garages'].fillna(df['garages'].mean())
    df['banos'] = df['banos'].fillna(df['banos'].mean())
    
    df['fecha'] = pd.to_datetime(df['fecha']).astype(int)

    
df_train = pd.read_csv('./data/train.csv')

# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train, df_test = features_de_csvs(df_train, df_test)



# df_train = utils.dolarizar_df(df_train)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)


features = ['habitaciones', 'garages', 'banos', 'fecha',
       'metroscubiertos', 'metrostotales', 'provincia',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['similares_count',
       'porcentaje_metros', 'diferencia_metros', 'intervalo_metros_totales',
       'intervalo_metros_cubiertos', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'tipo_propiedad_compartida', 'prop_frecuente', 'zona', 'top_provincia',
       'es_ciudad_centrica', 'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'anio', 'mes', 'dia',
       'trimestre', 'escualas_centros_cercanos', 'delincuencia', 'turismo',
       'es_antigua', 'cantidad_inquilinos', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'promedio_precio_ciudad_gen',
       'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona',
       'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_hbg_tipo_propiedad_provincia',
       'promedio_precio_booleanos', 'puntaje']

features_cat = ['provincia', 'tipodepropiedad', 'intervalo_metros_totales', 'intervalo_metros_cubiertos',
               'zona', 'escualas_centros_cercanos']

features += features_test + features_cat


features_a_normalizar = [f for f in features if f not in features_cat + ['precio']]

df_full = pd.concat([df_test_f, df_train_f.drop('precio', axis=1)], ignore_index=True)
df_full_g = utils.filtrar_features(df_full, features, 'precio')

df_full_ohe = pd.get_dummies(df_full_g, columns=features_cat)

df_train_n = df_full_ohe.iloc[len(df_test_f.values):len(df_full_ohe.values)].reset_index()
df_train_n['precio'] = df_train_f['precio']
df_test_n = df_full_ohe.iloc[0:len(df_test_f.values)].reset_index()



x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_n, 'precio', test_size=0.05)

model = Sequential()
model.add(BatchNormalization())
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.1)

model.fit(x_train.values, y_train.values, epochs=15)

y_pred_train = model.predict(x_train)

keras_mae_train = utils.MAE(y_train, y_pred_train)
print(f"MAE Keras (train): {keras_mae_train:.5f}")

In [ ]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)

In [ ]:

def normalizar_df(df, features, min_max=None):
    if min_max is None:
        min_max = MinMaxScaler()
        df[features] = pd.DataFrame(min_max.fit_transform(df[features]), columns=features)
        return df, min_max
    else:
        df[features] = pd.DataFrame(min_max.transform(df[features]), columns=features)
        return df, min_max
    

def llenar_nulls(df):
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metrostotales'].fillna(df['metrostotales'])
    
    df['habitaciones'] = df['habitaciones'].fillna(df['habitaciones'].mean())
    df['garages'] = df['garages'].fillna(df['garages'].mean())
    df['banos'] = df['banos'].fillna(df['banos'].mean())
    
    df['fecha'] = pd.to_datetime(df['fecha']).astype(int)


llenar_nulls(df_train_f)
llenar_nulls(df_test_f)


features = ['habitaciones', 'garages', 'banos', 'fecha',
       'metroscubiertos', 'metrostotales', 'provincia',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['similares_count',
       'porcentaje_metros', 'diferencia_metros', 'intervalo_metros_totales',
       'intervalo_metros_cubiertos', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'tipo_propiedad_compartida', 'prop_frecuente', 'zona', 'top_provincia',
       'es_ciudad_centrica', 'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'anio', 'mes', 'dia',
       'trimestre', 'escualas_centros_cercanos', 'delincuencia', 'turismo',
       'es_antigua', 'cantidad_inquilinos', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'promedio_precio_ciudad_gen',
       'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona',
       'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_hbg_tipo_propiedad_provincia',
       'promedio_precio_booleanos', 'puntaje']

features_cat = ['provincia', 'tipodepropiedad', 'intervalo_metros_totales', 'intervalo_metros_cubiertos',
               'zona', 'escualas_centros_cercanos']

features += features_test + features_cat


features_a_normalizar = [f for f in features if f not in features_cat + ['precio']]

df_train_g = utils.filtrar_features(df_train_f, features, 'precio')
df_train_n = pd.get_dummies(df_train_g, columns=features_cat)
df_train_n, min_max = normalizar_df(df_train_n, features_a_normalizar)

df_test_z = pd.concat([df_test_f, df_train_f.drop('precio', axis=1)], ignore_index=True, sort=False)
df_test_g = utils.filtrar_features(df_test_z, features)
df_test_n = pd.get_dummies(df_test_g, columns=features_cat).iloc[0:60000]
df_test_n, min_max = normalizar_df(df_test_n, features_a_normalizar, min_max)


In [ ]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)


In [ ]:
full_display(df_test_n.describe())

In [ ]:
y_pred_eval = model.predict(df_test_n)
df_test_f['target'] = y_pred_eval

# df_test_f = utils.pesificar_df(df_test_f, 'target', 'target')
df_test_f[['id', 'target']].to_csv('respuesta2-keras.csv', index=False)